In [1]:
from pathlib import Path
from dotenv import load_dotenv

if Path(".env").exists():
    load_dotenv()


In [2]:
import os
import pins

connect_url = os.environ["CONNECT_SERVER"]
connect_api_key = os.environ["CONNECT_API_KEY"]

board = pins.board_connect(server_url=connect_url, api_key=connect_api_key)

vessel_history_path = board.pin_download("sam.edwardes/vessel_history_clean")
vessel_verbose_path = board.pin_download("sam.edwardes/vessel_verbose_clean")
weather_path = board.pin_download("sam.edwardes/terminal_weather_clean")


In [3]:
from posit.connect import Client

with Client(url=connect_url, api_key=connect_api_key) as client:
    username = client.me.username
print(username)


michael.beigelmacher


In [4]:
import polars as pl

vessel_history = pl.read_parquet(vessel_history_path)
vessel_verbose = pl.read_parquet(vessel_verbose_path)
weather = pl.read_parquet(weather_path)


In [5]:
weather.glimpse()


Rows: 777120
Columns: 17
$ latitude                            <f64> 48.541298, 48.541298, 48.541298, 48.541298, 48.541298, 48.541298, 48.541298, 48.541298, 48.541298, 48.541298
$ longitude                           <f64> -122.727264, -122.727264, -122.727264, -122.727264, -122.727264, -122.727264, -122.727264, -122.727264, -122.727264, -122.727264
$ terminal_name                       <str> 'anacortes', 'anacortes', 'anacortes', 'anacortes', 'anacortes', 'anacortes', 'anacortes', 'anacortes', 'anacortes', 'anacortes'
$ generationtime_ms                   <f64> 0.47409534454345703, 0.47409534454345703, 0.47409534454345703, 0.47409534454345703, 0.47409534454345703, 0.47409534454345703, 0.47409534454345703, 0.47409534454345703, 0.47409534454345703, 0.47409534454345703
$ utc_offset_seconds                  <i64> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
$ timezone                            <str> 'GMT', 'GMT', 'GMT', 'GMT', 'GMT', 'GMT', 'GMT', 'GMT', 'GMT', 'GMT'
$ timezone_abbreviation              

In [6]:
vessel_history.glimpse()


Rows: 520619
Columns: 7
$ Vessel                        <str> 'cathlamet', 'cathlamet', 'cathlamet', 'cathlamet', 'cathlamet', 'cathlamet', 'cathlamet', 'cathlamet', 'cathlamet', 'cathlamet'
$ Departing                     <str> 'vashon island', 'fauntleroy', 'vashon island', 'fauntleroy', 'vashon island', 'southworth', 'vashon island', 'fauntleroy', 'southworth', 'vashon island'
$ Arriving                      <str> 'fauntleroy', 'vashon island', 'fauntleroy', 'vashon island', 'southworth', 'vashon island', 'fauntleroy', 'southworth', 'vashon island', 'fauntleroy'
$ ScheduledDepart <datetime[μs, UTC]> 2020-01-01 13:40:00+00:00, 2020-01-01 14:10:00+00:00, 2020-01-01 14:35:00+00:00, 2020-01-01 15:05:00+00:00, 2020-01-01 15:30:00+00:00, 2020-01-01 15:55:00+00:00, 2020-01-01 16:15:00+00:00, 2020-01-01 16:45:00+00:00, 2020-01-01 17:20:00+00:00, 2020-01-01 17:40:00+00:00
$ ActualDepart    <datetime[μs, UTC]> 2020-01-01 13:41:12+00:00, 2020-01-01 14:11:37+00:00, 2020-01-01 14:37:37+00:00, 20

In [7]:
ferry_trips = vessel_history.select(
    pl.col("Vessel", "Departing", "Arriving"),
    (pl.col("ActualDepart") - pl.col("ScheduledDepart"))
    .dt.total_seconds()
    .alias("Delay"),
    pl.col("Date"),
    pl.col("Date").dt.year().alias("Year"),
    pl.col("Date").dt.month().alias("Month"),
    pl.col("Date").dt.weekday().alias("Weekday"),
    pl.col("Date").dt.hour().alias("Hour"),
)
ferry_trips


Vessel,Departing,Arriving,Delay,Date,Year,Month,Weekday,Hour
str,str,str,i64,"datetime[μs, UTC]",i32,i8,i8,i8
"""cathlamet""","""vashon island""","""fauntleroy""",72,2020-01-01 13:40:00 UTC,2020,1,3,13
"""cathlamet""","""fauntleroy""","""vashon island""",97,2020-01-01 14:10:00 UTC,2020,1,3,14
"""cathlamet""","""vashon island""","""fauntleroy""",157,2020-01-01 14:35:00 UTC,2020,1,3,14
"""cathlamet""","""fauntleroy""","""vashon island""",131,2020-01-01 15:05:00 UTC,2020,1,3,15
"""cathlamet""","""vashon island""","""southworth""",61,2020-01-01 15:30:00 UTC,2020,1,3,15
…,…,…,…,…,…,…,…,…
"""yakima""","""friday harbor""","""lopez island""",2025,2024-06-07 01:25:00 UTC,2024,6,5,1
"""yakima""","""lopez island""","""anacortes""",2940,2024-06-07 02:10:00 UTC,2024,6,5,2
"""yakima""","""anacortes""","""lopez island""",2305,2024-06-07 03:25:00 UTC,2024,6,5,3


In [8]:
ferry_trips.plot.hist("Delay", bin_range=(-1800, 7200), bins=30)


%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


:Histogram   [Delay]   (Delay_count)

In [9]:
ferry_trips.select(
    pl.col("Delay")
    .map_elements(lambda x: max(x, 1), return_dtype=pl.Float64)
    .log()
    .alias("LogDelay")
).plot.hist("LogDelay")


:Histogram   [LogDelay]   (LogDelay_count)

In [10]:
ferry_trips = ferry_trips.select(
    pl.exclude("Delay"),
    pl.col("Delay")
    .map_elements(lambda x: max(x, 1), return_dtype=pl.Float64)
    .log()
    .alias("LogDelay"),
)
ferry_trips


Vessel,Departing,Arriving,Date,Year,Month,Weekday,Hour,LogDelay
str,str,str,"datetime[μs, UTC]",i32,i8,i8,i8,f64
"""cathlamet""","""vashon island""","""fauntleroy""",2020-01-01 13:40:00 UTC,2020,1,3,13,4.276666
"""cathlamet""","""fauntleroy""","""vashon island""",2020-01-01 14:10:00 UTC,2020,1,3,14,4.574711
"""cathlamet""","""vashon island""","""fauntleroy""",2020-01-01 14:35:00 UTC,2020,1,3,14,5.056246
"""cathlamet""","""fauntleroy""","""vashon island""",2020-01-01 15:05:00 UTC,2020,1,3,15,4.875197
"""cathlamet""","""vashon island""","""southworth""",2020-01-01 15:30:00 UTC,2020,1,3,15,4.110874
…,…,…,…,…,…,…,…,…
"""yakima""","""friday harbor""","""lopez island""",2024-06-07 01:25:00 UTC,2024,6,5,1,7.613325
"""yakima""","""lopez island""","""anacortes""",2024-06-07 02:10:00 UTC,2024,6,5,2,7.986165
"""yakima""","""anacortes""","""lopez island""",2024-06-07 03:25:00 UTC,2024,6,5,3,7.742836


In [11]:
vessel_verbose.glimpse()


Rows: 21
Columns: 44
$ VesselID                                   <i64> 1, 2, 65, 74, 15, 17, 52, 18, 19, 25
$ VesselSubjectID                            <i64> 1, 2, 428, 487, 15, 17, 52, 18, 19, 25
$ VesselName                                 <str> 'cathlamet', 'chelan', 'chetzemoka', 'chimacum', 'issaquah', 'kaleetan', 'kennewick', 'kitsap', 'kittitas', 'puyallup'
$ VesselAbbrev                               <str> 'cat', 'che', 'chz', 'chm', 'iss', 'kal', 'ken', 'kis', 'kit', 'puy'
$ ClassID                                    <i64> 10, 10, 162, 100, 10, 50, 162, 10, 10, 90
$ ClassName                                  <str> 'issaquah 130', 'issaquah 130', 'kwa-di tabil', 'olympic', 'issaquah 130', 'super', 'kwa-di tabil', 'issaquah 130', 'issaquah 130', 'jumbo mark ii'
$ ClassSubjectID                             <i64> 310, 310, 427, 319, 310, 314, 427, 310, 310, 318
$ DrawingImg                                 <str> 'https://www.wsdot.wa.gov/ferries/images/pages/boat_drawings/4-issaq

In [12]:
vessel_verbose


VesselID,VesselSubjectID,VesselName,VesselAbbrev,ClassID,ClassName,ClassSubjectID,DrawingImg,PublicDisplayName,SilhouetteImg,SortSeq,Status,OwnedByWSF,CarDeckRestroom,CarDeckShelter,Elevator,ADAAccessible,MainCabinGalley,MainCabinRestroom,PublicWifi,ADAInfo,AdditionalInfo,VesselNameDesc,VesselHistory,CityBuilt,SpeedInKnots,EngineCount,Horsepower,MaxPassengerCount,PassengerOnly,FastFerry,PropulsionInfo,TallDeckClearance,RegDeckSpace,TallDeckSpace,Tonnage,Displacement,YearBuilt,YearRebuilt,SolasCertified,MaxPassengerCountForInternational,BeamInches,LengthInches,DraftInches
i64,i64,str,str,i64,str,i64,str,str,str,i64,i64,bool,bool,bool,bool,bool,bool,bool,bool,str,str,str,str,str,i64,i64,i64,i64,bool,bool,str,i64,i64,i64,i64,i64,datetime[ms],datetime[ms],bool,f64,i32,i32,f64
1,1,"""cathlamet""","""cat""",10,"""issaquah 130""",310,"""https://www.wsdot.wa.gov/ferri…","""Issaquah""","""https://www.wsdot.wa.gov/ferri…",40,1,true,true,false,true,true,true,true,false,"""The MV Cathlamet has elevator …",null,"""From the Kathlamet tribe, the …",null,"""seattle, wa""",16,2,5000,1200,false,false,"""diesel""",186,124,26,2477,3310,1981-01-01 00:00:00,1993-01-01 00:00:00,false,null,944,3936,198.0
2,2,"""chelan""","""che""",10,"""issaquah 130""",310,"""https://www.wsdot.wa.gov/ferri…","""Issaquah""","""https://www.wsdot.wa.gov/ferri…",40,1,true,true,false,true,true,true,true,false,"""The MV Chelan has elevator acc…",null,"""From the Chelan language: Tsi…",null,"""seattle, wa""",16,2,5000,1200,false,false,"""diesel""",188,124,30,2477,3405,1981-01-01 00:00:00,2005-01-01 00:00:00,true,1090.0,944,3936,201.0
65,428,"""chetzemoka""","""chz""",162,"""kwa-di tabil""",427,"""https://www.wsdot.wa.gov/ferri…","""Kwa-di Tabil""","""https://www.wsdot.wa.gov/ferri…",75,1,true,false,false,true,true,true,true,false,"""MV Chetzemoka has elevator acc…",null,"""The name honors a friendly Nat…",null,"""seattle""",15,2,6000,748,false,false,"""diesel""",192,64,9,4623,2415,2010-01-01 00:00:00,null,false,null,768,3284,132.0
74,487,"""chimacum""","""chm""",100,"""olympic""",319,"""https://www.wsdot.wa.gov/ferri…","""Olympic""","""https://www.wsdot.wa.gov/ferri…",35,1,true,true,true,true,true,true,true,false,"""The vessel has two ADA complia…",null,"""“The Chimacum People who spoke…","""Chimacum is the third of the 1…","""seattle, wa""",17,2,6000,1500,false,false,"""diesel""",192,144,34,3525,4384,2017-01-01 00:00:00,null,false,null,998,4347,216.0
15,15,"""issaquah""","""iss""",10,"""issaquah 130""",310,"""https://www.wsdot.wa.gov/ferri…","""Issaquah""","""https://www.wsdot.wa.gov/ferri…",40,1,true,true,false,true,true,true,true,false,"""The MV Issaquah has elevator a…",null,"""""Snake."" Native Americans who …",null,"""seattle, wa""",16,2,5000,1200,false,false,"""diesel""",188,124,26,2475,3310,1979-01-01 00:00:00,1989-01-01 00:00:00,false,null,944,3936,198.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
33,33,"""tillikum""","""til""",20,"""evergreen state""",311,"""https://www.wsdot.wa.gov/ferri…","""Evergreen State""","""https://www.wsdot.wa.gov/ferri…",60,1,true,false,false,true,true,true,true,false,"""The MV Tillikum has an elevato…",null,"""Chinook Jargon: ""friends; rela…",null,"""seattle, wa""",13,2,2500,1061,false,false,"""diesel-electric (ac)""",162,87,30,2070,2413,1959-01-01 00:00:00,1994-01-01 00:00:00,false,null,878,3722,null
68,462,"""tokitae""","""tok""",100,"""olympic""",319,"""https://www.wsdot.wa.gov/ferri…","""Olympic""","""https://www.wsdot.wa.gov/ferri…",35,1,true,false,true,true,true,true,true,false,"""The vessel has two ADA complia…",null,"""Tokitae means ""nice day, prett…","""Tokitae is the first of the 14…","""seattle, wa""",17,2,6000,1500,false,false,"""diesel""",192,144,34,3525,4384,2014-01-01 00:00:00,null,false,null,998,4347,216.0
36,36,"""walla walla""","""wal""",70,"""jumbo""",316,"""https://www.wsdot.wa.gov/ferri…","""Jumbo""","""https://www.wsdot.wa.gov/ferri…",20,1,true,false,false,true,true,tru

In [13]:
ferry_info = vessel_verbose.select(
    pl.col("VesselName").str.to_lowercase(),
    pl.col("ClassName"),
    pl.col(
        "SpeedInKnots",
        "EngineCount",
        "Horsepower",
        "MaxPassengerCount",
        "PassengerOnly",
        "FastFerry",
        "PropulsionInfo",
    ),
    pl.col("YearBuilt", "YearRebuilt").dt.year(),
)

ferry_trips = ferry_trips.join(
    ferry_info, left_on="Vessel", right_on="VesselName", how="left", coalesce=True
)


In [14]:
import polars.selectors as cs

ferry_trips = (
    ferry_trips.with_columns(pl.col("Date").dt.round("1h").alias("time"))
    .join(
        weather.rename(lambda col_name: f"departing_{col_name}"),
        how="left",
        left_on=["Departing", "time"],
        right_on=["departing_terminal_name", "departing_time"],
        coalesce=True,
    )
    .join(
        weather.rename(lambda col_name: f"arriving_{col_name}"),
        how="left",
        left_on=["Arriving", "time"],
        right_on=["arriving_terminal_name", "arriving_time"],
        coalesce=True,
    )
    .select(
        ~cs.ends_with(
            "latitude",
            "longitude",
            "generationtime_ms",
            "utc_offset_seconds",
            "timezone",
            "timezone_abbreviation",
            "elevation",
            "hourly_units",
        ),
    )
    .select(pl.exclude("time"))
)
ferry_trips


Vessel,Departing,Arriving,Date,Year,Month,Weekday,Hour,LogDelay,ClassName,SpeedInKnots,EngineCount,Horsepower,MaxPassengerCount,PassengerOnly,FastFerry,PropulsionInfo,YearBuilt,YearRebuilt,departing_weather_code,departing_temperature_2m,departing_precipitation,departing_cloud_cover,departing_wind_speed_10m,departing_wind_direction_10m,departing_wind_gusts_10m,arriving_weather_code,arriving_temperature_2m,arriving_precipitation,arriving_cloud_cover,arriving_wind_speed_10m,arriving_wind_direction_10m,arriving_wind_gusts_10m
str,str,str,"datetime[μs, UTC]",i32,i8,i8,i8,f64,str,i64,i64,i64,i64,bool,bool,str,i32,i32,i64,f64,f64,i64,f64,i64,f64,i64,f64,f64,i64,f64,i64,f64
"""cathlamet""","""vashon island""","""fauntleroy""",2020-01-01 13:40:00 UTC,2020,1,3,13,4.276666,"""issaquah 130""",16,2,5000,1200,false,false,"""diesel""",1981,1993,3,9.2,0.0,87,26.2,226,53.3,3,9.1,0.0,92,31.6,222,55.1
"""cathlamet""","""fauntleroy""","""vashon island""",2020-01-01 14:10:00 UTC,2020,1,3,14,4.574711,"""issaquah 130""",16,2,5000,1200,false,false,"""diesel""",1981,1993,3,9.1,0.0,92,31.6,222,55.1,3,9.2,0.0,87,26.2,226,53.3
"""cathlamet""","""vashon island""","""fauntleroy""",2020-01-01 14:35:00 UTC,2020,1,3,14,5.056246,"""issaquah 130""",16,2,5000,1200,false,false,"""diesel""",1981,1993,51,8.9,0.1,60,26.7,212,56.5,2,8.7,0.0,77,34.6,211,63.4
"""cathlamet""","""fauntleroy""","""vashon island""",2020-01-01 15:05:00 UTC,2020,1,3,15,4.875197,"""issaquah 130""",16,2,5000,1200,false,false,"""diesel""",1981,1993,2,8.7,0.0,77,34.6,211,63.4,51,8.9,0.1,60,26.7,212,56.5
"""cathlamet""","""vashon island""","""southworth""",2020-01-01 15:30:00 UTC,2020,1,3,15,4.110874,"""issaquah 130""",16,2,5000,1200,false,false,"""diesel""",1981,1993,1,8.8,0.0,20,18.8,238,54.4,1,8.7,0.0,20,18.8,238,54.4
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""yakima""","""friday harbor""","""lopez island""",2024-06-07 01:25:00 UTC,2024,6,5,1,7.613325,"""super""",17,4,8000,2000,false,false,"""diesel-electric (dc)""",1967,2000,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""yakima""","""lopez island""","""anacortes""",2024-06-07 02:10:00 UTC,2024,6,5,2,7.986165,"""super""",17,4,8000,2000,false,false,"""diesel-electric (dc)""",1967,2000,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""yakima""","""anacortes""","""lopez island""",2024-06-07 03:25:00 UTC,2024,6,5,3,7.742836,"""super""",17,4,8000,2000,false,false,"""diesel-electric (dc)""",1967,2000,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [15]:
import polars.selectors as cs

ferry_trips.select(cs.string()).drop("Vessel")


Departing,Arriving,ClassName,PropulsionInfo
str,str,str,str
"""vashon island""","""fauntleroy""","""issaquah 130""","""diesel"""
"""fauntleroy""","""vashon island""","""issaquah 130""","""diesel"""
"""vashon island""","""fauntleroy""","""issaquah 130""","""diesel"""
"""fauntleroy""","""vashon island""","""issaquah 130""","""diesel"""
"""vashon island""","""southworth""","""issaquah 130""","""diesel"""
…,…,…,…
"""friday harbor""","""lopez island""","""super""","""diesel-electric (dc)"""
"""lopez island""","""anacortes""","""super""","""diesel-electric (dc)"""
"""anacortes""","""lopez island""","""super""","""diesel-electric (dc)"""


In [16]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_features = [
    "Month",
    "Weekday",
    "Hour",
    "SpeedInKnots",
    "EngineCount",
    "Horsepower",
    "MaxPassengerCount",
    "PassengerOnly",
    "FastFerry",
    "YearBuilt",
    "YearRebuilt",
    "departing_temperature_2m",
    "departing_precipitation",
    "departing_cloud_cover",
    "departing_wind_speed_10m",
    "departing_wind_direction_10m",
    "departing_wind_gusts_10m",
    "arriving_temperature_2m",
    "arriving_precipitation",
    "arriving_cloud_cover",
    "arriving_wind_speed_10m",
    "arriving_wind_direction_10m",
    "arriving_wind_gusts_10m",
]
categorical_features = [
    "Departing",
    "Arriving",
    "ClassName",
    "PropulsionInfo",
    "departing_weather_code",
    "arriving_weather_code",
]


preprocessor = ColumnTransformer(
    [("num", "passthrough", numeric_features), ("cat", OneHotEncoder(), categorical_features)]
)


In [17]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(verbose=2, random_state=2, n_jobs=-1)


In [18]:
from sklearn.pipeline import Pipeline

model = Pipeline([("preprocess", preprocessor), ("random-forest", rf)])


In [19]:
from sklearn.model_selection import train_test_split

X = ferry_trips.drop_nulls().drop("Vessel", "Date", "Year", "LogDelay")
y = ferry_trips.drop_nulls()["LogDelay"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
print(f"Nrows training data: {X_train.shape[0]}")
print(f"Nrows testing data:  {X_test.shape[0]}")


Nrows training data: 190176
Nrows testing data:  47545


In [20]:
model.fit(X_train.to_pandas(), y_train)
model.score(X_test, y_test)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


building tree 1 of 100building tree 2 of 100

building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100


[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    6.5s


building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   27.4s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.3s finished


0.2912894603014551

In [21]:
from vetiver import VetiverModel

v = VetiverModel(model, model_name=f"{username}/ferry_delay", prototype_data=X.to_pandas())


In [22]:
import pins
import vetiver

model_board = pins.board_connect(server_url=connect_url, api_key=connect_api_key, allow_pickle_read=True)
vetiver.vetiver_pin_write(model_board, model=v)


Model Cards provide a framework for transparent, responsible reporting. 
 Use the vetiver `.qmd` Quarto template as a place to start, 
 with vetiver.model_card()
Writing pin:
Name: 'michael.beigelmacher/ferry_delay'
Version: 20240626T161959Z-16815


In [23]:
from rsconnect.api import RSConnectServer

connect_server = RSConnectServer(url=connect_url, api_key=connect_api_key)
vetiver.deploy_rsconnect(connect_server=connect_server, board=model_board, pin_name=f"{username}/ferry_delay")


             Consider creating a requirements.txt file instead.


/Users/michael/git/ds-workflows-python/materials/03-model-training/.venv/lib/python3.12/site-packages/vetiver/write_fastapi.py:21: UserWarning: Pinned vetiver model has no active version and no datetime on versions,
              Do you need to check your pinned model?
              Using version 555571
  warnings.warn(
Validating server... 	[OK]
Validating app mode... 	[OK]
Making bundle ... 	[OK]
Deploying bundle ... 	[OK]
Saving deployed information... 	[OK]
Building FastAPI application...
Bundle created with Python version 3.12.3 is compatible with environment Local with Python version 3.12.3 from /opt/python/3.12.3/bin/python3.12 
Bundle requested Python version 3.12.3; using /opt/python/3.12.3/bin/python3.12 which has version 3.12.3
2024/06/26 20:24:02.783428470 [rsc-session] Content GUID: 823c479e-3d5e-4898-8801-a5c2cec97bb5
2024/06/26 20:24:02.784018849 [rsc-session] Content ID: 12154
2024/06/26 20:24:02.784046650 [rsc-session] Bundle ID: 555572
2024/06/26 20:24:02.784051171 [r